# Core Imports

In [1]:
# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Logging
from tqdm import tqdm as tqdm_text
from tqdm.notebook import tqdm as tqdm_notebook

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology, Molecule

from openforcefields.openforcefields import get_forcefield_dirs_paths
OPENFF_DIR = Path(get_forcefield_dirs_paths()[0])

# File and chemistry type definitions

In [2]:
topo_dir = Path('Topologies')
topo_dir.mkdir(exist_ok=True)

# lammps_dir = Path('LAMMPS')
lammps_dir = Path('LAMMPS')
lammps_dir.mkdir(exist_ok=True)

omm_dir = Path('OpenMM')
omm_dir.mkdir(exist_ok=True)

In [3]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Creating OpenMM and LAMMPS systems

## Generating Interchange dict

In [4]:
# specify forcefield
# ff_name = 'openff-2.0.0.offxml'
ff_name = 'openff_unconstrained-2.0.0.offxml'
ff_path = OPENFF_DIR / ff_name
forcefield = ForceField(ff_path)

# Interchange generation
success_ics = defaultdict(defaultdict)
failed_ics  = defaultdict(list)

for chemistry in chemistries:
    chem_dir = topo_dir / chemistry
    
    progress = tqdm_notebook([path for path in chem_dir.iterdir()]) # unpack into list for progress bar
    for sdf_path in progress:
        mol_name = sdf_path.stem
        progress.set_postfix_str(f'{chemistry} : {mol_name}')

        try:
            offmol = Molecule.from_file(sdf_path, allow_undefined_stereo=True)
            offtop = Topology.from_molecules(offmol) 
            ic = forcefield.create_interchange(offtop, charge_from_molecules=[offmol])
            success_ics[chemistry][mol_name] = ic
        except Exception as e:
            print(e)
            failed_ics[e.__class__.__name__].append(sdf_path)

for err_name, err_list in failed_ics.items():
    for sdf_path in err_list:
        sdf_path.unlink() # delete dud files

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

Unable to read molecule from file: Topologies/urethane/urethane_38.sdf
BondHandler was not able to find parameters for the following valence terms:

- Topology indices (476, 2064): names and elements ( H), ( H), 
- Topology indices (504, 2144): names and elements ( H), ( H), 
- Topology indices (620, 2456): names and elements ( H), ( H), 
- Topology indices (525, 2185): names and elements ( H), ( H), 
- Topology indices (536, 2215): names and elements ( H), ( H), 
- Topology indices (184, 1236): names and elements ( H), ( H), 
- Topology indices (616, 2452): names and elements ( H), ( H), 
- Topology indices (647, 2522): names and elements ( H), ( H), 
- Topology indices (964, 3425): names and elements ( H), ( H), 
- Topology indices (806, 2988): names and elements ( H), ( H), 
- Topology indices (157, 1163): names and elements ( H), ( H), 
- Topology indices (641, 2515): names and elements ( H), ( H), 
- Topology indices (727, 2759): names and elements ( H), ( H), 
- Topology indices 

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 38, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 39, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 60, aromatic: False, chiral: False
Atom atomic num: 7, name: , idx: 105, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 102, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 106, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 127, aromatic: False, chiral: False
Atom atomic num: 7, name: , idx: 172, aromatic: False, chiral: True with bonds:
bond order: 1, chiral:

Unable to read molecule from file: Topologies/urethane/urethane_22.sdf
Unable to read molecule from file: Topologies/urethane/urethane_45.sdf
BondHandler was not able to find parameters for the following valence terms:

- Topology indices (260, 1454): names and elements ( H), ( H), 
- Topology indices (484, 2077): names and elements ( H), ( H), 
- Topology indices (129, 1074): names and elements ( H), ( H), 
- Topology indices (914, 3284): names and elements ( H), ( H), 
- Topology indices (297, 1543): names and elements ( H), ( H), 
- Topology indices (639, 2500): names and elements ( H), ( H), 
- Topology indices (537, 2214): names and elements ( H), ( H), 
- Topology indices (431, 1912): names and elements ( H), ( H), 
- Topology indices (517, 2156): names and elements ( H), ( H), 
- Topology indices (158, 1162): names and elements ( H), ( H), 
- Topology indices (910, 3271): names and elements ( H), ( H), 
- Topology indices (590, 2378): names and elements ( H), ( H), 
- Topology i

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): urethane_9
Problematic atoms are:
Atom atomic num: 7, name: , idx: 23, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 22, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 24, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 31, aromatic: True, chiral: False
Atom atomic num: 7, name: , idx: 69, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 68, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 70, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 77, aromatic: True, chiral: False
Atom atomic num: 7, name: , idx: 115, aromatic: False, chiral: True with bonds:
bond order: 1, c

Unable to read molecule from file: Topologies/urethane/urethane_27.sdf
Unable to read molecule from file: Topologies/urethane/urethane_0.sdf


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): urethane_12
Problematic atoms are:
Atom atomic num: 7, name: , idx: 34, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 33, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 42, aromatic: True, chiral: False
Atom atomic num: 7, name: , idx: 91, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 90, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 92, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 99, aromatic: True, chiral: False
Atom atomic num: 7, name: , idx: 148, aromatic: False, chiral: True with bonds:
bond order: 1, 

Unable to read molecule from file: Topologies/urethane/urethane_30.sdf
Unable to read molecule from file: Topologies/urethane/urethane_11.sdf


## Defining utility functions

In [5]:
from openmm import XmlSerializer
from openmm import System, Context, State
from openmm import Integrator, Force
from openmm.app import Simulation
from openmm.unit import nanometer

from openff.interchange import Interchange
from openff.units import unit as offunit


DEFAULT_STATE_PARAMS : dict[str, bool] = {
    'getPositions'  : True,
    'getVelocities' : True,
    'getForces'     : True,
    'getEnergy'     : True,
    'getParameters' : True,
    'getParameterDerivatives' : False,
    'getIntegratorParameters' : False
}


def serialize_state_and_sys(sim : Simulation, out_dir : Path, out_name : str, state_params : dict[str, bool]=DEFAULT_STATE_PARAMS) -> None:
    '''For saving State and System info of a Simulation to disc'''
    sim_dict = {
        'system' : sim.system,
        'state' : sim.context.getState(**state_params)
    }
    
    for affix, save_data in sim_dict.items():
        save_path = out_dir / f'{out_name}_{affix}.xml'
        save_path.touch()

        with save_path.open('w') as file:
            file.write( XmlSerializer.serialize(save_data) )

def apply_state_to_context(state : State, context : Context) -> None:
    '''For applying saved State data to an existing OpenMM Simulation'''
    context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
    context.setPositions(state.getPositions())
    context.setVelocities(state.getVelocities())
    context.setTime(state.getTime())

    context.reinitialize(preserveState=True)    

def load_openmm_system(sys_path : Path, extra_forces : Optional[Union[Force, Iterable[Force]]]=None, sep_force_grps : bool=True, remove_constrs : bool=False) -> System:
    '''Load and configure a serialized OpenMM system, with optional additional parameters'''
    assert(sys_path.suffix == '.xml')
    with sys_path.open('r') as file:
        ommsys = XmlSerializer.deserialize(file.read())

    if extra_forces: # deliberately sparse to handle both Nonetype and empty list
        for force in extra_forces: 
            ommsys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(ommsys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(ommsys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            ommsys.removeConstraint(i)

    return ommsys

def create_simulation2(interchange : Interchange, integrator : Integrator, forces : Optional[Iterable[Force]]=None,
                        sep_force_grps : bool=True, remove_constrs : bool=True, combine_nonbonded_forces : bool=True) -> Simulation:
    '''Specifies configuration for an OpenMM Simulation - Interchange load alows many routes for creation'''
    openmm_sys = interchange.to_openmm(combine_nonbonded_forces=combine_nonbonded_forces) 
    openmm_top = interchange.topology.to_openmm()
    openmm_pos = interchange.positions.m_as(offunit.nanometer) * nanometer

    if forces: # deliberately sparse to handle both Nonetype and empty list
        for force in forces: 
            openmm_sys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(openmm_sys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(openmm_sys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            openmm_sys.removeConstraint(i)

    simulation = Simulation(openmm_top, openmm_sys, integrator)
    simulation.context.setPositions(openmm_pos)

    return simulation

## Manually create OpenMM sims from Interchange

In [6]:
# specifying simulation and ensemble parameters
from openff.interchange.components.mdconfig import MDConfig

from openmm.app import Simulation
from openmm import NonbondedForce, CustomNonbondedForce
from openmm import MonteCarloBarostat, LangevinMiddleIntegrator

from openmm.unit import atmosphere, kelvin, nanometer
from openmm.unit import femtosecond, picosecond
from openff.units import unit as offunit

# Box sizes
BOX_VECS = np.eye(3) * 10 * nanometer

# Long-range parameters
# CUTOFF = 2.0 * nanometer
CUTOFF = 0.9 * nanometer
# CUTOFF_METHOD = NonbondedForce.NoCutoff
# CUTOFF_METHOD = NonbondedForce.CutoffNonPeriodic
CUTOFF_METHOD = NonbondedForce.CutoffPeriodic

DISPERSION = True
SWITCHING  = False

# Thermodynamic/integrator parameters
T = 300*kelvin
P = 1*atmosphere

timestep = 2*femtosecond
friction = 1*picosecond**-1

# ======================================

force_names = (
    'vdW',
    'Electrostatic',
    'vdW 1-4',
    'Electrostatic 1-4',
    'Dihedral',
    'Angle',
    'Bond'
)

# looping over all urethanes
omm_sims = defaultdict(defaultdict)
for chemistry, ic_dict in success_ics.items():
    lmp_chem_dir = lammps_dir / chemistry
    lmp_chem_dir.mkdir(exist_ok=True)
    
    omm_chem_dir = omm_dir/ chemistry
    omm_chem_dir.mkdir(exist_ok=True)

    progress = tqdm_notebook(ic_dict.items())
    for mol_name, interchange in progress:
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        
    # creating OpenMM Simulation
        progress.set_description('Building OpenMM Simulation')
        # specifying thermo/baro to determine ensemble
        integrator = LangevinMiddleIntegrator(T, friction, timestep)
        # extra_forces = [MonteCarloBarostat(P, T, baro_freq)]
        extra_forces = None

        # loading OpenMM sim components from Interchange
        interchange.box = BOX_VECS
        omm_top = interchange.topology.to_openmm()
        omm_sys = interchange.to_openmm(combine_nonbonded_forces=False)
        omm_pos = interchange.positions.m_as(offunit.nanometer)

        ## Setting box vectors for periodic forces
        omm_top.setPeriodicBoxVectors(BOX_VECS)
        omm_sys.setDefaultPeriodicBoxVectors(*BOX_VECS)

        # configuring bound Force objects
        if extra_forces:
            for force in extra_forces:
                omm_sys.addForce(force)

        ## number all forces into separate force groups for separability
        for i, force in enumerate(omm_sys.getForces()):
            force.setForceGroup(i)

        ## Add labels to default forces
        for force, name in zip(omm_sys.getForces(), force_names):
            force.setName(name)

        ## reconfiguring non-bonded forces
        ### Custom nonbonded
        # nonbond_custom = omm_sys.getForce(0)
        # assert(isinstance(nonbond_custom, CustomNonbondedForce))

        # nonbond_custom.setCutoffDistance(CUTOFF)
        # nonbond_custom.setUseSwitchingFunction(SWITCHING)
        # nonbond_custom.setNonbondedMethod(CUTOFF_METHOD)
        # nonbond_custom.setUseLongRangeCorrection(DISPERSION)
 
        # ### Default nonbonded
        # nonbond = omm_sys.getForce(1)
        # assert(isinstance(nonbond, NonbondedForce))

        # nonbond.setCutoffDistance(CUTOFF)
        # nonbond.setNonbondedMethod(CUTOFF_METHOD)
        # nonbond.setUseSwitchingFunction(SWITCHING)
        # nonbond.setUseDispersionCorrection(DISPERSION)

        # create OpenMM Simulation
        sim = Simulation(omm_top, omm_sys, integrator)
        sim.context.setPositions(omm_pos)
        omm_sims[chemistry][mol_name] = sim

    # saving OpenMM files
        progress.set_description('Generating OpenMM files')
        omm_mol_dir = omm_chem_dir / mol_name
        omm_mol_dir.mkdir(exist_ok=True)

        sdf_out_path = omm_mol_dir / f'{mol_name}_topology.sdf'
        sdf_out_path.touch()

        for mol in interchange.topology.molecules: # use OpenFF format for saving Molecules (much more convenient to work with)
            mol.to_file(str(sdf_out_path), file_format=sdf_out_path.suffix[1:])
        serialize_state_and_sys(sim, out_dir=omm_mol_dir, out_name=mol_name)

    # saving LAMMPS files
        progress.set_description('Generating LAMMPS files')
        lmp_mol_dir = lmp_chem_dir / mol_name
        lmp_mol_dir.mkdir(exist_ok=True)

        lmp_path = lmp_mol_dir / f'{mol_name}.lammps'
        lmp_in_path = lmp_mol_dir / f'{mol_name}.in'

        ### creating .lmp file
        lmp = interchange.to_lammps(lmp_path)
        mdc = MDConfig.from_interchange(interchange)
        mdc.write_lammps_input(lmp_in_path)

        ### creating .in file, replacing input file with .lmp from above
        with lmp_in_path.open('r') as in_file:
            in_file_block = in_file.read()

        in_file_block = in_file_block.replace('out.lmp', str(lmp_path))

        with lmp_in_path.open('w') as in_file:
            in_file.write(in_file_block)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

# Evaluating LAMMPS energies

In [7]:
ENERGY_EVAL_INP = Path('in.urethane') # path to 

E_MAP = {
    'ebond'  : 'Bond',
    'eangle' : 'Angle',
    'edihed' : 'Proper Torsion',
    'eimp'   : 'Improper Torsion',
    'ecoul'  : 'Coulomb Short',
    'elong'  : 'Coulomb Long',
    'evdwl'  : 'vdW',
    'etail'  : 'Dispersion',
    'epair'  : 'Nonbonded',
    'pe'     : 'Potential',
    'ke'     : 'Kinetic',
    'etotal' : 'Total'
}

CELL_KW = ( # keywords for probing unit cell sizes and angles
    'cella',
    'cellb',
    'cellc',
    'cellalpha',
    'cellbeta',
    'cellgamma',
)

In [8]:
def get_calc_lmp_energies(lmp_block : str) -> tuple[str, list[str]]:
    '''Read which thermodynamic energy contributions will be calculated from a LAMMPS input file block'''
    ENERGY_CONTRIB_REGEX = re.compile(r'^thermo_style\s(?P<thermo_style>\b\w*?\b)\s(?P<calc_energies>.*$)')

    for line in lmp_block.split('\n'):
        if (match := re.search(ENERGY_CONTRIB_REGEX, line)):
            groups = match.groupdict()
            return groups['thermo_style'], groups['calc_energies'].split(' ')
    else:
        raise ValueError('No thermo_style energy commands found in input file')

In [9]:
import lammps
from IPython.display import clear_output


records = {}
cell_sizes = {}
for subdir in lammps_dir.iterdir():
    if subdir.is_dir():
        chemistry = subdir.name
        for mol_dir in subdir.iterdir():
            mol_name = mol_dir.stem
            lammps_file = mol_dir / f'{mol_name}.lammps'
            lammps_in   = mol_dir / f'{mol_name}.in'
            
            # craete LAMMPS wrapper and execute input calc
            with lammps.lammps() as lmp: # need to create new lammps() object instance for each run
                # lmp.commands_string( ENERGY_EVAL_STR.replace('$INP_FILE', str(lammps_file)) )
                # lmp.file(str(ENERGY_EVAL_INP))
                lmp.file(str(lammps_in)) # read input file and calculate energies

                ## Getting energies
                with lammps_in.open('r') as in_file:
                    thermo_style, calc_energies = get_calc_lmp_energies(in_file.read())

                energies = {
                    E_MAP[contrib] : lmp.get_thermo(contrib)
                        for contrib in calc_energies
                }

                ## Getting unit cell dimensions
                cell_params = {
                    cp : lmp.get_thermo(cp)
                        for cp in CELL_KW
                }

            # reformatting energies
            energies = {
                f'{contrib} (kcal/mol)' : energy # add units to labels
                    for contrib, energy in energies.items()
            }
            
            # save records for Pandas DataFrames
            records[(chemistry, mol_name)] = energies
            cell_sizes[(chemistry, mol_name)] = cell_params
            clear_output() # wipe lengthy LAMMPS printouts

In [10]:
lmp_table = pd.DataFrame.from_dict(records, 'index')
lmp_table.index.names  = ['Chemistry', 'Molecule'] # ensure index labels are labelled consistently
lmp_table.sort_values('Molecule', inplace=True)
lmp_table.to_csv(lammps_dir/f'{lammps_dir.name}_PEs.csv')

# Evaluating OpenMM energies

## Loading simulations from file

In [11]:
skip = True

# parameters
sep_force_grps : bool = True
remove_constrs : bool = False

# iterate over serialized directory tree and load
if not skip:
    omm_sims = defaultdict(defaultdict)
    for subdir in omm_dir.iterdir():
        if subdir.is_dir():
            chemistry = subdir.name
            for mol_dir in subdir.iterdir():
                mol_name = mol_dir.name

                state_file = mol_dir / f'{mol_name}_state.xml'
                sys_file   = mol_dir / f'{mol_name}_system.xml'
                top_file   = mol_dir / f'{mol_name}_topology.sdf'

                offmol = Molecule.from_file(top_file)
                offtop = Topology.from_molecules(offmol)
                
                integrator = LangevinMiddleIntegrator(T, friction, timestep)
                # extra_forces = [MonteCarloBarostat(P, T, baro_freq)]
                extra_forces = None

                # load and configure System
                omm_top = offtop.to_openmm()
                omm_sys = load_openmm_system(
                    sys_file,
                    extra_forces=extra_forces,
                    sep_force_grps=sep_force_grps,
                    remove_constrs=remove_constrs
                )

                # putting it all together into a Simulation
                sim = Simulation(
                    topology=omm_top,
                    system=omm_sys,
                    integrator=integrator,
                    state=state_file
                )
                omm_sims[chemistry][mol_name] = sim

## Evaluating starting structure energies

In [12]:
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole

NULL_ENERGY = 0.0*kilojoule_per_mole
PRECISION : int = 4

data_dicts = []
for chemistry, mol_dict in omm_sims.items():
    progress = tqdm_notebook(mol_dict.items())
    for mol_name, sim in progress:
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        
        # extract total and component energies from OpenMM force groups
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }
        omm_energies = {}

        ## Total Potential
        overall_state = sim.context.getState(getEnergy=True) # get total potential energy
        PE = overall_state.getPotentialEnergy()
        omm_energies['Potential'] = PE

        ## Total Kinetic (to verify no integration is being done)
        KE = overall_state.getKineticEnergy()
        omm_energies['Kinetic'] = KE
        assert(KE == NULL_ENERGY)

        ## Individual force contributions
        for i, force in enumerate(sim.system.getForces()):
            state = sim.context.getState(getEnergy=True, groups={i})
            omm_energies[force.getName()] = state.getPotentialEnergy()

        # reformat to desired units and precision
        omm_energies_kcal = {}
        for contrib_name, energy_kj in omm_energies.items():
            energy_kcal = energy_kj.in_units_of(kilocalorie_per_mole)
            omm_energies_kcal[f'{contrib_name} ({energy_kcal.unit.get_symbol()})'] = round(energy_kcal._value, PRECISION)

        # compile data
        data_dict = {**data_dict, **omm_energies_kcal}
        data_dicts.append(data_dict)

omm_table = pd.DataFrame.from_records(data_dicts)
omm_table.sort_values('Molecule', inplace=True)
omm_table.set_index(['Chemistry', 'Molecule'], inplace=True)
omm_table.to_csv(omm_dir / f'{omm_dir.name}_PEs.csv')

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

# Comparing energies

## Loading energy tables and comparing contributions

In [13]:
pd.options.display.float_format = '{:.4f}'.format # disable scientific notation

@dataclass
class TableFormats:
    table_key : str
    sum_terms : dict[str, list[str]]
    del_terms : list[str]

omm_formats = TableFormats(
    table_key = omm_dir.stem,
    sum_terms = {
        'vdW (kcal/mol)' : ['vdW (kcal/mol)', 'vdW 1-4 (kcal/mol)'],
        'Coulomb (kcal/mol)' : ['Electrostatic (kcal/mol)', 'Electrostatic 1-4 (kcal/mol)']
    },
    del_terms = ['Kinetic (kcal/mol)']
)

lmp_formats = TableFormats(
    table_key = lammps_dir.stem,
    sum_terms = {
        'vdW (kcal/mol)' : ['vdW (kcal/mol)', 'Dispersion (kcal/mol)'],
        'Dihedral (kcal/mol)' : ['Proper Torsion (kcal/mol)', 'Improper Torsion (kcal/mol)'],
        'Coulomb (kcal/mol)' : ['Coulomb Short (kcal/mol)', 'Coulomb Long (kcal/mol)']
    },
    del_terms = ['Nonbonded (kcal/mol)']
)

# apply reformatting to respective tables
for fmt in (omm_formats, lmp_formats):
    table_in_path  = Path(fmt.table_key) / f'{fmt.table_key}_PEs.csv'
    table_out_path = Path(fmt.table_key) / f'{fmt.table_key}_PEs_processed.csv'
    table = pd.read_csv(table_in_path, index_col=(0, 1)).sort_index(axis=1)

    # combine selected terms
    for combined_contrib, contribs in fmt.sum_terms.items():
        new_term = sum(
            table[contrib]
                for contrib in contribs
        ) # merge contributions into a single new named term
        table.drop(columns=contribs, inplace=True) # clear contributions
        table[combined_contrib] = new_term # done after drop to ensure name clashes don;t result in extra deletion
    
    # delete redundant terms
    for del_contrib in fmt.del_terms:
        table.drop(columns=[del_contrib], inplace=True) # clear contributions

    globals()[f'{fmt.table_key.lower()}_table'] = table
    table.to_csv(table_out_path)

In [14]:
openmm_table

Angle (kcal/mol)  Bond (kcal/mol)  Dihedral (kcal/mol)  \
Chemistry Molecule                                                              
NIPU      NIPU_10             1464.9790        1177.0361             315.7242   
          NIPU_11              940.1827        1039.7645             276.4011   
          NIPU_12              936.3028        1033.4167             300.9877   
          NIPU_2              1725.1020        1050.6371             390.4701   
          NIPU_8               994.7223         939.9698             312.5088   
          NIPU_9               953.5693        1030.6256             281.7358   
urethane  urethane_1           725.4003         841.3459             156.8210   
          urethane_10         1070.3106         633.0817             465.9121   
          urethane_12          741.1011         936.4742             210.0150   
          urethane_13          575.8502         717.7118             112.4079   
          urethane_14         1328.1954         650.5202             282.7600   
          urethane_15          816.9968         629.7148             276.2996   
          urethane_16          897.4787         644.1676             194.8601   
          urethane_17         1741.5982        1521.4752             326.8120   
          urethane_18          718.5248         629.9913             170.9781   
          urethane_19         1329.0353         651.3659             277.7959   
          urethane_2          1307.5052         647.7079             219.9248   
          urethane_21         2074.3112        1829.8451             396.4748   
          urethane_23          494.7744         712.3935              53.9121   
          urethane_24          710.6854         616.4662             177.6603   
          urethane_26          634.5888         624.8095             228.5454   
          urethane_29          599.1136         822.2865             177.7349   
          urethane_3           409.0155         594.1615              91.3075   
          urethane_31          888.5591         655.2848             233.4105   
          urethane_32          676.5683         627.5007             143.0098   
          urethane_33          489.8905         599.0784             160.3545   
          urethane_34          977.0853         628.2897             187.7550   
          urethane_35          642.9720         938.2825              72.7398   
          urethane_36         1137.2077         636.1647             519.3126   
          urethane_37          716.0589         619.4731             199.6749   
          urethane_39          725.5664         625.8189             198.3839   
          urethane_40          632.5229         612.4052             150.9659   
          urethane_41         1772.6337         678.9329             328.0828   
          urethane_42         1735.8002        1538.8286             231.4421   
          urethane_43         1223.0334         653.8066             484.0010   
          urethane_44          996.7395        1131.6646             267.4205   
          urethane_46          536.0869         603.6788             127.2272   
          urethane_47         1158.8372         965.5097             290.4110   
          urethane_48         2006.7206        1653.4150             378.2049   
          urethane_49         1458.0015         720.0151             235.8394   
          urethane_5          1036.4399         625.9435             434.5786   
          urethane_50         1287.4210         646.3993             179.1081   
          urethane_51          790.8953         617.6525             111.5041   
          urethane_52          884.4422         623.4035             186.2730   
          urethane_53          802.3654         634.4806             204.0338   
          urethane_54          625.1730         613.7287             166.6244   
          urethane_55          627.7131         609.0258             282.3683   
          urethane_56          577.0522         615.36

In [15]:
lammps_table

Angle (kcal/mol)  Bond (kcal/mol)  \
Chemistry Molecule                                         
NIPU      NIPU_10             1464.9796        1177.0406   
          NIPU_11              940.1828        1039.7616   
          NIPU_12              936.3032        1033.4171   
          NIPU_2              1725.1022        1050.6351   
          NIPU_8               994.7213         939.9715   
          NIPU_9               953.5694        1030.6327   
urethane  urethane_0          4417.8420         745.0806   
          urethane_1           725.4000         841.3442   
          urethane_10         1070.3101         633.0824   
          urethane_12          741.1011         936.4751   
          urethane_13          575.8516         717.7127   
          urethane_14         1328.1960         650.5216   
          urethane_15          816.9970         629.7169   
          urethane_16          897.4788         644.1677   
          urethane_17         1741.5982        1521.4719   
          urethane_18          718.5236         629.9915   
          urethane_19         1329.0359         651.3634   
          urethane_2          1307.5048         647.7087   
          urethane_21         2074.3111        1829.8459   
          urethane_22         6848.7400         713.5584   
          urethane_23          494.7748         712.3986   
          urethane_24          710.6857         616.4658   
          urethane_26          634.5880         624.8085   
          urethane_29          599.1137         822.2865   
          urethane_3           409.0151         594.1612   
          urethane_31          888.5598         655.2840   
          urethane_32          676.5674         627.4996   
          urethane_33          489.8900         599.0831   
          urethane_34          977.0856         628.2903   
          urethane_35          642.9715         938.2892   
          urethane_36         1137.2086         636.1628   
          urethane_37          716.0597         619.4700   
          urethane_39          725.5658         625.8221   
          urethane_40          632.5238         612.4068   
          urethane_41         1772.6362         678.9303   
          urethane_42         1735.8008        1538.8258   
          urethane_43         1223.0351         653.8017   
          urethane_44          996.7395        1131.6647   
          urethane_45         4355.3228         722.5881   
          urethane_46          536.0867         603.6815   
          urethane_47         1158.8364         965.5119   
          urethane_48         2006.7219        1653.4197   
          urethane_49         1458.0017         720.0157   
          urethane_5          1036.4398         625.9430   
          urethane_50         1287.4208         646.4005   
          urethane_51          790.8958         617.6500   
          urethane_52          884.4428         623.4099   
          urethane_53          802.3663         634.4781   
          urethane_54          625.1740         613.7273   
          urethane_55          627.7133         609.0287   
          urethane_56          577.0524         615.3628   
          urethane_58          615.8726         606.8114   
          urethane_59          515.1033         817.6024   
          urethane_6          1116.3392         937.7560   
          urethane_60          531.8400         602.0459   
          urethane_61          447.9992         596.3474   
          urethane_62          902.8854         647.2763   
          urethane_7           658.9548         607.2377   
          urethane_8           710.0555         611.9162   
          urethane_9           592.8018         710.7793   

                       Potential (kcal/mol)       vdW (kcal/mol)  \
Chemistry Molecule                                                 
NIPU      NIPU_10                43620.7571           41700.6257   
          NIPU_11              3418654.3262         3416884.9484   
          NIPU_12             

In [16]:
diff = openmm_table - lammps_table
diff

Angle (kcal/mol)  Bond (kcal/mol)  Coulomb (kcal/mol)  \
Chemistry Molecule                                                             
NIPU      NIPU_10               -0.0006          -0.0045              4.6007   
          NIPU_11               -0.0001           0.0029              1.6560   
          NIPU_12               -0.0004          -0.0004              3.4446   
          NIPU_2                -0.0002           0.0020              5.0905   
          NIPU_8                 0.0010          -0.0017              2.1054   
          NIPU_9                -0.0001          -0.0071              5.8012   
urethane  urethane_0                NaN              NaN                 NaN   
          urethane_1             0.0003           0.0017              2.3808   
          urethane_10            0.0005          -0.0007             -0.6811   
          urethane_12            0.0000          -0.0009              4.3647   
          urethane_13           -0.0014          -0.0009              1.7259   
          urethane_14           -0.0006          -0.0014              0.7418   
          urethane_15           -0.0002          -0.0021             -0.1789   
          urethane_16           -0.0001          -0.0001              1.3786   
          urethane_17            0.0000           0.0033             -1.6457   
          urethane_18            0.0012          -0.0002             -0.1527   
          urethane_19           -0.0006           0.0025             -0.5959   
          urethane_2             0.0004          -0.0008              1.0044   
          urethane_21            0.0001          -0.0008             -1.9946   
          urethane_22               NaN              NaN                 NaN   
          urethane_23           -0.0004          -0.0051             -1.0283   
          urethane_24           -0.0003           0.0004             -1.3635   
          urethane_26            0.0008           0.0010              1.1947   
          urethane_29           -0.0001           0.0000              5.1640   
          urethane_3             0.0004           0.0003              1.6143   
          urethane_31           -0.0007           0.0008              2.9847   
          urethane_32            0.0009           0.0011             -0.3237   
          urethane_33            0.0005          -0.0047             -0.4112   
          urethane_34           -0.0003          -0.0006             -5.6052   
          urethane_35            0.0005          -0.0067              1.9710   
          urethane_36           -0.0009           0.0019              4.9958   
          urethane_37           -0.0008           0.0031              1.3727   
          urethane_39            0.0006          -0.0032             -0.8201   
          urethane_40           -0.0009          -0.0016              4.0217   
          urethane_41           -0.0025           0.0026             -0.2321   
          urethane_42           -0.0006           0.0028              1.4359   
          urethane_43           -0.0017           0.0049              1.6456   
          urethane_44            0.0000          -0.0001              4.7844   
          urethane_45               NaN              NaN                 NaN   
          urethane_46            0.0002          -0.0027              1.6963   
          urethane_47            0.0008          -0.0022              2.0442   
          urethane_48           -0.0013          -0.0047             -0.1500   
          urethane_49           -0.0002          -0.0006              0.5324   
          urethane_5             0.0001           0.0005              3.5010   
          urethane_50            0.0002          -0.0012             -0.4400   
          urethane_51           -0.0005           0.0025              7.5689   
          urethane_52           -0.0006          -0.0064              1.4718   
          urethane_53           -0.0009           0.0025             -0.3839   
          urethane_54  

In [17]:
common_cols = ['Angle (kcal/mol)', 'Bond (kcal/mol)']# 'Torsion (kcal/mol)']

omm_redux = omm_table.drop(columns=common_cols)
lmp_redux = lmp_table.drop(columns=common_cols)

In [18]:
omm_table[common_cols] - lmp_table[common_cols]

Angle (kcal/mol)  Bond (kcal/mol)
Chemistry Molecule                                      
NIPU      NIPU_10               -0.0006          -0.0045
          NIPU_11               -0.0001           0.0029
          NIPU_12               -0.0004          -0.0004
          NIPU_2                -0.0002           0.0020
          NIPU_8                 0.0010          -0.0017
          NIPU_9                -0.0001          -0.0071
urethane  urethane_0                NaN              NaN
          urethane_1             0.0003           0.0017
          urethane_10            0.0005          -0.0007
          urethane_12            0.0000          -0.0009
          urethane_13           -0.0014          -0.0009
          urethane_14           -0.0006          -0.0014
          urethane_15           -0.0002          -0.0021
          urethane_16           -0.0001          -0.0001
          urethane_17            0.0000           0.0033
          urethane_18            0.0012          -0.0002
          urethane_19           -0.0006           0.0025
          urethane_2             0.0004          -0.0008
          urethane_21            0.0001          -0.0008
          urethane_22               NaN              NaN
          urethane_23           -0.0004          -0.0051
          urethane_24           -0.0003           0.0004
          urethane_26            0.0008           0.0010
          urethane_29           -0.0001           0.0000
          urethane_3             0.0004           0.0003
          urethane_31           -0.0007           0.0008
          urethane_32            0.0009           0.0011
          urethane_33            0.0005          -0.0047
          urethane_34           -0.0003          -0.0006
          urethane_35            0.0005          -0.0067
          urethane_36           -0.0009           0.0019
          urethane_37           -0.0008           0.0031
          urethane_39            0.0006          -0.0032
          urethane_40           -0.0009          -0.0016
          urethane_41           -0.0025           0.0026
          urethane_42           -0.0006           0.0028
          urethane_43           -0.0017           0.0049
          urethane_44            0.0000          -0.0001
          urethane_45               NaN              NaN
          urethane_46            0.0002          -0.0027
          urethane_47            0.0008          -0.0022
          urethane_48           -0.0013          -0.0047
          urethane_49           -0.0002          -0.0006
          urethane_5             0.0001           0.0005
          urethane_50            0.0002          -0.0012
          urethane_51           -0.0005           0.0025
          urethane_52           -0.0006          -0.0064
          urethane_53           -0.0009           0.0025
          urethane_54           -0.0010           0.0014
          urethane_55           -0.0002          -0.0029
          urethane_56           -0.0002           0.0002
          urethane_58           -0.0006          -0.0005
          urethane_59            0.0000          -0.0002
          urethane_6            -0.0008           0.0005
          urethane_60           -0.0001           0.0006
          urethane_61            0.0002           0.0016
          urethane_62            0.0000          -0.0002
          urethane_7            -0.0008          -0.0016
          urethane_8            -0.0007          -0.0021
          urethane_9            -0.0010           0.0029

In [19]:
omm_redux

Potential (kcal/mol)  Kinetic (kcal/mol)  \
Chemistry Molecule                                                
NIPU      NIPU_10                43627.0660              0.0000   
          NIPU_11              3418594.7646              0.0000   
          NIPU_12                 2129.6208              0.0000   
          NIPU_2              64613163.8853              0.0000   
          NIPU_8                  2994.6808              0.0000   
          NIPU_9                576201.7948              0.0000   
urethane  urethane_1              3775.9851              0.0000   
          urethane_10             1843.4328              0.0000   
          urethane_12            17941.8348              0.0000   
          urethane_13             1784.6714              0.0000   
          urethane_14           149199.3548              0.0000   
          urethane_15        635327190.4291              0.0000   
          urethane_16         10648090.0800              0.0000   
          urethane_17      76048515319.6603              0.0000   
          urethane_18             3527.4599              0.0000   
          urethane_19      23697417574.8029              0.0000   
          urethane_2           4010178.2094              0.0000   
          urethane_21     197280004264.8726              0.0000   
          urethane_23             2551.6797              0.0000   
          urethane_24    2562604959797.6909              0.0000   
          urethane_26            18461.6821              0.0000   
          urethane_29              684.4724              0.0000   
          urethane_3          30035864.1994              0.0000   
          urethane_31         12921601.7451              0.0000   
          urethane_32        592253921.0350              0.0000   
          urethane_33    3433711226521.3882              0.0000   
          urethane_34      23990107466.6760              0.0000   
          urethane_35             2740.2250              0.0000   
          urethane_36             1830.6109              0.0000   
          urethane_37             4086.9329              0.0000   
          urethane_39             7066.3608              0.0000   
          urethane_40             3335.4961              0.0000   
          urethane_41           150478.2631              0.0000   
          urethane_42      77019454800.9201              0.0000   
          urethane_43    2355304191342.7725              0.0000   
          urethane_44         22228429.1542              0.0000   
          urethane_46           180105.9546              0.0000   
          urethane_47             2004.0543              0.0000   
          urethane_48             1106.3023              0.0000   
          urethane_49             3609.0344              0.0000   
          urethane_5     2052738263347.1152              0.0000   
          urethane_50  387506180884839.3750              0.0000   
          urethane_51      23939432269.7676              0.0000   
          urethane_52      23859198013.2580              0.0000   
          urethane_53           344287.1607              0.0000   
          urethane_54           345417.6007              0.0000   
          urethane_55           182280.2287              0.0000   
          urethane_56        123199790.7907              0.0000   
          urethane_58           349735.0150              0.0000   
          urethane_59            95349.1778              0.0000   
          urethane_6          22218983.7961              0.0000   
          urethane_60          2938408.0190              0.0000   
          urethane_61     678791687026.9127              0.0000   
          urethane_62             2047.0165              0.0000   
          urethane_7            242315.3208              0.0000   
          urethane_8       23907609992.4161              0.0000   
          urethane_9             17884.4590              0.0000   

                            vdW (kcal/mol)  Electrostatic (kcal/mol) 

## Evaluating energies with drivers

In [20]:
from openff.interchange.drivers.openmm import get_openmm_energies, _get_openmm_energies
from openff.interchange.drivers.lammps import get_lammps_energies, _get_lammps_energies,  _find_lammps_executable
from openff.units.openmm import to_openmm as openff_units_to_openmm

In [21]:
{
    contrib : openff_units_to_openmm(value).in_units_of(kilocalorie_per_mole)
        for contrib, value in get_openmm_energies(interchange, detailed=True, combine_nonbonded_forces=False).energies.items()
}

{'vdW': Quantity(value=23988746576.956284, unit=kilocalorie/mole),
 'Electrostatics': Quantity(value=110.10667610613562, unit=kilocalorie/mole),
 'vdW 1-4': Quantity(value=473.62292642905527, unit=kilocalorie/mole),
 'Electrostatics 1-4': Quantity(value=-686.2929735580487, unit=kilocalorie/mole),
 'Torsion': Quantity(value=187.7549812559763, unit=kilocalorie/mole),
 'Angle': Quantity(value=977.0855728882693, unit=kilocalorie/mole),
 'Bond': Quantity(value=628.290289497926, unit=kilocalorie/mole)}

In [22]:
get_lammps_energies(interchange).energies

{'Bond': 628.29029 <Unit('kilocalorie_per_mole')>,
 'Angle': 977.08558 <Unit('kilocalorie_per_mole')>,
 'Torsion': 187.75496789099998 <Unit('kilocalorie_per_mole')>,
 'vdW': 23988754999.810043 <Unit('kilocalorie_per_mole')>,
 'Electrostatics': -570.5814300000001 <Unit('kilocalorie_per_mole')>}

## Comparing ParmEd energy decomposition to native OpenMM force-group-based decomposition

In [23]:
import parmed
from openmm.openmm import Force

NULL_ENERGY = 0.0*kilojoule_per_mole

sim = omm_sims['urethane']['urethane_41']
# assign and initialize unique force groups for simulation
for i, force in enumerate(sim.system.getForces()):
    force.setForceGroup(i)
    # print(force.getName(), force.getForceGroup())
sim.context.reinitialize(preserveState=True) # need to reinitialize to get force labelling changes to "stick"

# energies from OpenMM force groups
print('\nOpenMM:')
print('='*30)
omm_energies = {}

## extract total energies for state
overall_state = sim.context.getState(getEnergy=True) # get total potential energy
PE = overall_state.getPotentialEnergy()
omm_energies['Total Potential Energy'] = PE

KE = overall_state.getKineticEnergy()
assert(KE == NULL_ENERGY)

for i, force in enumerate(sim.system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    force_name = force.getName().removesuffix('Force')
    pe = state.getPotentialEnergy()

    omm_energies[force_name] = pe
    print(f'{force_name} : {pe}')

## converting name to match with ParmEd for comparison
namemap = {
    'Nonbonded' : 'bond',
    'PeriodicTorsion' : 'angle',
    'HarmonicAngle' : 'dihedral',
    'HarmonicBond' : 'urey_bradley',
    'Total Potential Energy' : 'total'
}
compat_omm_energies = {
    namemap[contrib] : energy
        for contrib, energy in omm_energies.items()
}

total = sum(omm_energies.values(), start=NULL_ENERGY) # need "seed" to have Quantity datatype to sum
print(f'{general.GREEK_UPPER["delta"]}E_contrib: ', PE - total)

# ParmEd energy decomposition
print('\nParmEd:')
print('='*30)
parm_energies = {}
parm_struct = parmed.openmm.load_topology(sim.topology, sim.system)
for contrib, energy_val in parmed.openmm.energy_decomposition(parm_struct, sim.context).items():
    parm_energies[contrib] = energy = energy_val*kilocalorie_per_mole # assign proper units
    print(contrib, energy.in_units_of(kilojoule_per_mole))


OpenMM:
vdW : 616436.7693361759 kJ/mol
Electrostatic : 681.4612459126311 kJ/mol
vdW 1-4 : 3828.738037109375 kJ/mol
Electrostatic 1-4 : -2975.945068359375 kJ/mol
Dihedral : 1372.698486328125 kJ/mol
Angle : 7416.69921875 kJ/mol
Bond : 2840.6552734375 kJ/mol


KeyError: 'vdW'